Sans fonction
types_colonnes = data.dtypes
dictionnaire_types = {col: str(dtype) for col, dtype in zip(data.columns, types_colonnes)}# pour avoir le nom de la colonne qui est de quelle type

dict_val_col = {}
for column, dtype in dictionnaire_types.items():
    if dtype == 'String':
        unique_values = data_gl.select(pl.col(column).unique()).to_series().to_list()#to_serie pour avoir un objet unidimentional et to_list pour pouvoir crée le dictionnaire notame,t pour pouvoir accéder au valeur de la cléf ce qui n'est pas possible avec un obj de tyoe serie
        dict_val_col[column] = unique_values
print(dict_val_col)

# Package

In [1]:
import polars as pl

In [2]:
import Final.lib1 as lib

# Fonction (réutilisable pour d'autre bases de donénes)

In [112]:
data_gl = pl.read_excel("IRENA_Stats_capacite_prod.xlsx",
                   sheet_name="Global")

### Fonction pour connaitre le nom de chaque colonne et leur type

In [47]:
def type_col(data):
    types_colonnes = data.dtypes
    dictionnaire_types = {col: str(dtype) for col, dtype in zip(data.columns, types_colonnes)}# pour avoir le nom de la colonne qui est de quelle type
    return dictionnaire_types


### Fonction pour avoir les catégorie possible pour les variable de type string

In [103]:
def categorie_col(data):
    dict_type_col=type_col(data)
    dict_val_col = {}
    for column, dtype in dict_type_col.items():
        if dtype == 'String':
            unique_values = data_gl.select(pl.col(column).unique()).to_series().to_list()#to_serie pour avoir un objet unidimentional et to_list pour pouvoir crée le dictionnaire notame,t pour pouvoir accéder au valeur de la cléf ce qui n'est pas possible avec un obj de tyoe serie
            dict_val_col[column] = unique_values
    return dict_val_col


In [84]:
test=categorie_col(data_gl)

In [78]:
nombre_elements = {clé: len(valeur) for clé, valeur in test.items()}

In [86]:
cles_moins_de_20 = [clé for clé, nombre in nombre_elements.items() if nombre < 20]

### Converstion des variable de type catégorielle

In [91]:
conversion_dict = {col: pl.Categorical for col in cles_moins_de_20}
with pl.StringCache():
    data_gl=data_gl.cast(conversion_dict)

### Séléction des énergies renouvlable seulement

A voir plus tard si les données sur les énergies non renouvlables pourrai npus être utile

In [138]:
data_total_renewable = data_gl.filter(pl.col("RE or Non-RE") == "Total Renewable")

### Fonction qui permet d'avoir un tabelau des valeurs manquantes en pourcentage

avec des extentions on peut le voir assez rapidement sur un explorateur de base de données + arrondir les valeur

In [25]:
def table_valeur_manquante(data):
    obs_manquantes = data.select([
    pl.col(obs).is_null().sum()
    for obs in data.columns])
    pct_obs_manquantes = (obs_manquantes / data.height * 100).transpose(include_header=True, header_name="Column")
    return pct_obs_manquantes


In [157]:
table_valeur_manquante(data_total_renewable)

Column,column_0
str,f64
"""RE or Non-RE""",0.0
"""Group Technology""",0.0
"""Year""",0.0
"""Sum of Electricity Installed C…",24.210526
"""Sum of Electricity Generation …",27.368421
"""Sum of Heat Generation (TJ)""",51.578947
"""Sum of Public Flows (2021 USD …",21.052632
"""Sum of SDG 7a1 Intl. Public Fl…",21.578947


### Séléction par group de technologie

In [100]:
def get_unique_values(data, column_name):
    return data.select(pl.col(column_name).unique()).to_series().to_list()

In [159]:
liste_technoloogie=get_unique_values(data_total_renewable,"Group Technology")

In [140]:
liste_technoloogie

['Bioenergy',
 'Other renewable energy',
 'Geothermal energy',
 'Wind energy',
 'Hydropower (excl. Pumped Storage)',
 'Solar energy',
 'Marine energy',
 'Multiple renewables*']

In [143]:
data_bio=data_total_renewable.filter(pl.col("Group Technology") == liste_technoloogie[0])

In [231]:
pct_obs_manquantes=pl.DataFrame()
for tech in liste_technoloogie:
    if tech == liste_technoloogie[0]:
        # Filtrer les données par technologie
        filtered_data = data_total_renewable.filter(pl.col("Group Technology") == tech)
        # Obtenir le tableau des valeurs manquantes pour cette technologie
        pct_obs_manquantes=table_valeur_manquante(filtered_data).rename({"column_0":str(tech)})
    else:
        filtered_data = data_total_renewable.filter(pl.col("Group Technology") == tech)
        t_val_m=table_valeur_manquante(filtered_data).rename({"column_0":str(tech)})
        t_val_m=t_val_m.select(tech)
        pl.concat([pct_obs_manquantes,t_val_m],how="horizontal")
        print(tech)

Multiple renewables*
Wind energy
Geothermal energy
Hydropower (excl. Pumped Storage)
Solar energy
Marine energy
Other renewable energy


In [237]:

# Initialiser un DataFrame vide pour commencer la concaténation
pct_obs_manquantes = pl.DataFrame()

for tech in liste_technoloogie:
    # Filtrer les données par technologie
    filtered_data = df_f.filter(pl.col("Group Technology") == tech)
    
    # Obtenir le tableau des valeurs manquantes pour cette technologie
    pct_obs = table_valeur_manquante(filtered_data)
    
    # Renommer la colonne pour refléter la technologie
    pct_obs = pct_obs.rename({"column_0": str(tech)})
    
    # Concaténer le DataFrame courant avec le DataFrame existant
    if pct_obs_manquantes.shape[0] == 0:
        pct_obs_manquantes = pct_obs
    else:
        pct_obs=pct_obs.select(tech)
        pct_obs_manquantes = pl.concat([pct_obs_manquantes, pct_obs], how="horizontal")

# Afficher le DataFrame final
print(pct_obs_manquantes)


shape: (8, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Column    ┆ Bioenergy ┆ Multiple  ┆ Wind      ┆ … ┆ Hydropowe ┆ Solar     ┆ Marine    ┆ Other    │
│ ---       ┆ ---       ┆ renewable ┆ energy    ┆   ┆ r (excl.  ┆ energy    ┆ energy    ┆ renewabl │
│ str       ┆ f64       ┆ s*        ┆ ---       ┆   ┆ Pumped    ┆ ---       ┆ ---       ┆ e energy │
│           ┆           ┆ ---       ┆ f64       ┆   ┆ Stora…    ┆ f64       ┆ f64       ┆ ---      │
│           ┆           ┆ f64       ┆           ┆   ┆ ---       ┆           ┆           ┆ f64      │
│           ┆           ┆           ┆           ┆   ┆ f64       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ RE or     ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ Non-RE    ┆           ┆           ┆           ┆   ┆           ┆           ┆

In [238]:
pct_obs_manquantes

Column,Bioenergy,Multiple renewables*,Wind energy,Geothermal energy,Hydropower (excl. Pumped Storage),Solar energy,Marine energy,Other renewable energy
str,f64,f64,f64,f64,f64,f64,f64,f64
"""RE or Non-RE""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Group Technology""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Year""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Sum of Electricity Installed C…",0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0
"""Sum of Electricity Generation …",0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0
"""Sum of Heat Generation (TJ)""",0.0,100.0,100.0,0.0,100.0,0.0,100.0,0.0
"""Sum of Public Flows (2021 USD …",0.0,0.0,0.0,4.347826,0.0,0.0,43.478261,100.0
"""Sum of SDG 7a1 Intl. Public Fl…",0.0,0.0,0.0,4.347826,0.0,0.0,47.826087,100.0


- A voir ce que signifie "mutiple renewables*" dans la documentation car les seul info dispo sont les flux
- Voir si les flux sont utilisable ou non car se base sur 2021 USD et on a déja des données sur des investisement il faudrai pouvoir voir ce qu'ils signifie exactement
- Pas de données sur la production d'électricité pour les énergiesr renouvlables autres

In [236]:
df_f = data_total_renewable.filter(pl.col("Year") != 2023)

In [240]:
df_fff=df_f.filter(pl.col('Group Technology')=="Bioenergy")

In [145]:
table_valeur_manquante(data_total_renewable.filter(pl.col("Group Technology") == liste_technoloogie[0])) #pour regarder modalité par modalité

Column,column_0
str,f64
"""RE or Non-RE""",0.0
"""Group Technology""",0.0
"""Year""",0.0
"""Sum of Electricity Installed C…",0.0
"""Sum of Electricity Generation …",4.166667
"""Sum of Heat Generation (TJ)""",100.0
"""Sum of Public Flows (2021 USD …",45.833333
"""Sum of SDG 7a1 Intl. Public Fl…",50.0


### Fonction pour convertir en categorie toutes les variable de type string (pour l'instant inutile)

In [140]:
conversion_dict = {col: pl.Categorical for col in colonne_string}
@pl.StringCache()
def fct(data):
    #data = data.cast({'RE or Non-RE': pl.Categorical})
    data=data.cast(conversion_dict)
    return data